In [55]:
from itertools import combinations
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
from summa import summarizer
from gensim.summarization import summarize
import networkx as nx
import numpy as np

from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()

In [87]:
with open('dataset_43428_1.txt') as file_data:
    result = []
    for text in file_data:
        if text in ['[\n', ']\n']:
            continue
        else:
            orig_text = '. '.join([' '.join(words) for words in split(text)])
            if len(orig_text) <= 300:
                result.append('"' + orig_text + '"')
                continue
            text = summarize(orig_text, word_count=300)
#             text += ' ' + orig_text
            text = text.replace('\n', ' ')
            text = text.replace('.', ' ')
            text = text.replace('  ', ' ')

            result.append('"' + text[:300] + '"')
#             break
            
with open('output.txt', 'w') as file_res:
    file_res.write("[" + ', '.join(result) + "]")

In [13]:
def split(text):
    tmp = []
    curr_sent = []
    curr_word = ''
    for ch in text:
        curr_word = curr_word.lower()
        if ch == ' ':
            if len(curr_word) > 0:
                if ad.only_alphabet_chars(curr_word, "CYRILLIC"):
                    curr_sent.append(curr_word)
                curr_word = ''
        elif ch.isalpha():
            if ch == 'n':
                if len(curr_word) > 1:
                    if ad.only_alphabet_chars(curr_word, "CYRILLIC"):
#                         print(curr_word)
                        curr_sent.append(curr_word)
                    curr_word = ''
                if len(curr_sent) > 1:
                    tmp.append(curr_sent)
                    curr_sent = []
            else:
                if ad.only_alphabet_chars(curr_word, "CYRILLIC"):
                    curr_word += ch

    if len(curr_word) > 0:
        if ad.only_alphabet_chars(curr_word, "CYRILLIC"):
            curr_sent.append(curr_word)
    if len(curr_sent) > 0:
        tmp.append(curr_sent)
    
#     tmp = tmp[:3] + tmp[-1]
    
    return tmp

In [9]:
def annot(text):        
    
    result = ''
    
    for i in range(1, min(4, len(text))):
        result += ' '.join(text[i]) + ' '
    
    result += ' '.join(text[-1]) + ' '
        
    return result[:300]

In [167]:
def annot(text):        
    words = {}

    for sent in text:
        for word in sent:
            word = word.lower()
            words[word] = words.get(word, 0) + 1
            
#     print(words)
            
    most_freq = max(list(words.values()))
    
    scores = np.zeros(len(text))

    for i, sent in enumerate(text):
        for j, word in enumerate(sent):
            word = word.lower()
            next_freq = None if j == len(sent) - 1 else words[sent[j + 1].lower()]
            if words[word] not in [1, most_freq]:
                scores[i] += words[word] * (1 if next_freq is None or next_freq == most_freq else next_freq)
        scores[i] /= len(sent)
        
    ind = np.argsort(scores)
    
    result = ''
    for i in ind:
        result += ' '.join(text[i]) + ' '
        
    return result[:300]